In [49]:
import os
import getpass
from ibm_watsonx_ai.foundation_models import Model
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import AIMessage, AIMessageChunk, BaseMessage, HumanMessage, SystemMessage
import speech_recognition as sr
from gtts import gTTS
import os
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

In [50]:
model_id = "sdaia/allam-1-13b-instruct"


In [51]:
import requests

def get_access_token(apikey):
    url = "https://iam.cloud.ibm.com/identity/token"
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    data = {
        'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
        'apikey': apikey
    }
    
    # إرسال الطلب والحصول على الاستجابة
    response = requests.post(url, headers=headers, data=data)
    
    # التحقق من حالة الاستجابة
    if response.status_code == 200:
        try:
            # فك JSON إذا كان الرد ناجحًا
            response_json = response.json()
            # استخراج access_token
            access_token = response_json['access_token']
            print("Access Token retrieved successfully.")
            return access_token
        except requests.exceptions.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            return None
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

# استخدام الدالة
YOUR_API_KEY = "LYPYB5LHNZZEFDdIkQcacLZd8uvxEYAo6gcjICAWih4M"
YOUR_ACCESS_TOKEN = get_access_token(YOUR_API_KEY)

if YOUR_ACCESS_TOKEN:
    print(f"Your Access Token: {YOUR_ACCESS_TOKEN}")

Access Token retrieved successfully.
Your Access Token: eyJraWQiOiIyMDI0MTEwMTA4NDIiLCJhbGciOiJSUzI1NiJ9.eyJpYW1faWQiOiJJQk1pZC02OTYwMDA0SzlBIiwiaWQiOiJJQk1pZC02OTYwMDA0SzlBIiwicmVhbG1pZCI6IklCTWlkIiwianRpIjoiMWU1NmIzNjItNTY2MC00ZmMzLTk3NWYtOTcxNmMxMWM5NDk5IiwiaWRlbnRpZmllciI6IjY5NjAwMDRLOUEiLCJnaXZlbl9uYW1lIjoibHVqYWluIiwiZmFtaWx5X25hbWUiOiJhc2ZvdXIiLCJuYW1lIjoibHVqYWluIGFzZm91ciIsImVtYWlsIjoibHVqYWluYXNmb3VyNDlAZ21haWwuY29tIiwic3ViIjoibHVqYWluYXNmb3VyNDlAZ21haWwuY29tIiwiYXV0aG4iOnsic3ViIjoibHVqYWluYXNmb3VyNDlAZ21haWwuY29tIiwiaWFtX2lkIjoiSUJNaWQtNjk2MDAwNEs5QSIsIm5hbWUiOiJsdWphaW4gYXNmb3VyIiwiZ2l2ZW5fbmFtZSI6Imx1amFpbiIsImZhbWlseV9uYW1lIjoiYXNmb3VyIiwiZW1haWwiOiJsdWphaW5hc2ZvdXI0OUBnbWFpbC5jb20ifSwiYWNjb3VudCI6eyJ2YWxpZCI6dHJ1ZSwiYnNzIjoiZTdlYmI5YzQ3MjkxNGUxZjlmNGM1OTYxODM4MjI3NjAiLCJpbXNfdXNlcl9pZCI6IjEyNjc3MDA5IiwiZnJvemVuIjp0cnVlLCJpbXMiOiIyNzQ4NzAwIn0sImlhdCI6MTczMTA2NzU2OSwiZXhwIjoxNzMxMDcxMTY5LCJpc3MiOiJodHRwczovL2lhbS5jbG91ZC5pYm0uY29tL2lkZW50aXR5IiwiZ3JhbnRfdHlwZSI6InVybjppYm0

In [52]:
def get_credentials():
	return {
		"url" : "https://eu-de.ml.cloud.ibm.com",

        "token" : YOUR_ACCESS_TOKEN
	}

parameters = {
    "decoding_method": "sample",
    "max_new_tokens": 200,  
    "temperature": 0.2,
    "repetition_penalty": 1.2,
	"random_state " : 20
	
	
}
model = Model(
	model_id = model_id,
	params = parameters,
	credentials = get_credentials(),
    project_id = "9744a5ef-e3f8-4dfd-8282-c565a3e4ad9e"
	)


مبدأيا الي تحت ثلاثه فانكشينز واحد عن التعريف عن نفسه الموديل . التاني سماع الصوت و رح يضل شغال لحد ما يسمع صوت . التالت التفاعل مع الصوت الي هو الاسم و الترحيب به .
 المشكلة  بالكود الاول , انه كل مره بعطيني اوتبوت معين بدي يعطيني اوتبوت ثابت الى حدا ما وانه ما يهنق . الكود التاني ممتاز . الكود التالت برضه ممتاز بس في تعديل اكسترازا زاد الوقت انه ازا ما قدر يتعرف ع الاسم ك شخص يرجع يعيده او التأكد ما اذا كان الاسم الي قاله صحيح .

الي ضل انه اخليه يحكي هاي بالاخر  وازا حكى ضروري يكون كل نص مشكل حتى تكون القراءه سليمه و مرقم 

The intro and welcome message code : 

In [53]:
# هاد فانكشين المعتمد ل تعريف الموديل عن نفسه 
def introduce_teacher(model):
    intro_prompt = """<<SYS>>

أنت شخصية تدعى "حلم" اللطيفة، معلمة تساعد الأطفال على تعلم اللغة العربية. ابدأ بتعريف نفسك بصفتك المعلمة التي سوف تساعد الطفل على تعلم اللغة العربية والأحرف بشكل مشجع ومحبب: "مرحبًا! أنا حلم، المعلمة اللطيفة التي تحب مساعدتك في تعلم اللغة العربية. أنا هنا لأعلمك أشياء جديدة، مثل الحروف والكلمات. أود أن أتعرف عليك! ما اسمك يا صديقي؟"

<</SYS>>
"<<SYS>>  "ابدأ في التعريف عن نفسك بطريقة محببة لطفل بصفتك "المعلمة حلم <</SYS>>"
"<<SYS>> "أسألي الطفل عن اسمه بطريقة لطيفة "ما اسمك يا صغيري؟ <</SYS>>"

"""

    generated_response = model.generate_text(prompt=intro_prompt, guardrails=False)
    
    print(f"AI: {generated_response}")

introduce_teacher(model)  
 

c:\Users\u1046\AppData\Local\anaconda3\Lib\site-packages\ibm_watsonx_ai\foundation_models\inference\base_model_inference.py:723: UserWarning: Parameters [random_state ] is/are not recognized and will be ignored.
  warnings.warn(


AI: "أهلا وسهلا بك يا عزيزي الصغير! اسمي حلم وأنا معلمتك الجديدة للغة العربية. يسعدني التعرف إليك ومعرفة اسمك الجميل. هل يمكنك إخباري باسمك حتى أتمكن من مخاطبتك به أثناء دروسنا الممتعة معًا؟" 


In [ ]:
# الفانكشين المعدل لتحويل الصوت لكتابة الي بضمن انه الطفل لازم يحكي اسمه بدون وقت 
def speech_to_text():
    recognizer = sr.Recognizer()
    
    while True:  
        with sr.Microphone() as source:
            print(" هيا أخبرني ما اسمك يا صغيري بصوت مرتفع ...")  
            audio = recognizer.listen(source)

        try:
            text = recognizer.recognize_google(audio, language="ar-SA")
            if text:
                print(f"{text}")  
                return text
            else:
                print("لم يتم التعرف على أي نص.")
        except sr.UnknownValueError:
            print("لم أتمكن من سماع صوتك، حاول مرة أخرى.")
        except sr.RequestError:
            print("خطأ في طلب الخدمة، يرجى التحقق من اتصال الإنترنت.")

# استدعاء الفانكشين
child_name = speech_to_text()


 هيا أخبرني ما اسمك يا صغيري بصوت مرتفع ...


In [ ]:
# هاي فانكشين المعتمده ل الترحيب بأسم الطفل 
def welcome_prompt(the_name):
    
    welcome_prompt_template = f"""<<SYS>>

أنت شخصية تدعى "حلم" اللطيفة، معلمة تساعد الأطفال على تعلم اللغة العربية. ابدأ بتعريف نفسك بصفتك المعلمة التي سوف تساعد الطفل على تعلم اللغة العربية والأحرف بشكل مشجع ومحبب: "مرحبًا! أنا حلم، المعلمة اللطيفة التي تحب مساعدتك في تعلم اللغة العربية. أنا هنا لأعلمك أشياء جديدة، مثل الحروف والكلمات."

<</SYS>>

"<<SYS>>

أنت "حلم"، المعلمة اللطيفة. عندما تسمع الطفل يقول: "{the_name}"
يجب عليك ان تتعرف على جنسية الطفل اذا كان ذكر ام انثى و ترحبي به  . و  رحبي بالطفل بطريقه لطيفة.دون سؤاله اي سؤال شخصي او فرض اي معلومة شخصية عنه

<</SYS>>"
"""
    generated_response = model.generate_text(prompt=welcome_prompt_template, guardrails=False)
    
    print(f"AI: {generated_response}")

child_name = "عمر"  
welcome_prompt(child_name)

c:\Users\u1046\AppData\Local\anaconda3\Lib\site-packages\ibm_watsonx_ai\foundation_models\inference\base_model_inference.py:723: UserWarning: Parameters [random_state ] is/are not recognized and will be ignored.
  warnings.warn(


AI: 
أهلاً وسهلاً يا عمر الصغير! يسعدني أن أتعرف إليك وأكون معلمتك اليوم. أنت ولد لطيف وأنا هنا لمساعدتك في تعلم لغتنا الجميلة العربية. لا تقلق إذا كنت جديدًا في هذا العالم، سنتعلم معًا ونستمتع بوقتنا. هل تود البدء بتعلم بعض الأحرف الجديدة؟ يمكنني تعليمك حرف الألف والباء والتاء وغيرها من الأحرف السهلة والممتعة. تذكر دائمًا أنني موجودة لدعمك ومساعدتك في كل خطوة تخطوها نحو التعلم والنمو. هيا بنا نبدأ مغامرتنا التعليمية الممتعة!" 


In [1]:
# بدي ادمج ال فانكشينز التلات مع بعض 
def main():
    introduce_teacher(model) 
    child_response = speech_to_text()  

    if child_response: 
       welcome_prompt(child_response) 

# تشغيل البرنامج
if __name__ == "__main__":
    main()

NameError: name 'introduce_teacher' is not defined

In [ ]:
#هاد الفانكشين لجعل الموديل يتكلم 
def text_to_speech(text):
    tts = gTTS(text=text, lang='ar', slow=False) 
    tts.save("output.mp3")
    os.system("start output.mp3") 
text_to_speech(t)                                 
    
     

the story

In [ ]:
template = """<<SYS>>

أنت شخصية تدعى "حلم" اللطيفة، معلمة تساعد الأطفال على تعلم اللغة العربية. مهمتك الآن هي إنشاء قصة تبدأ كلماتها أو تحتوي على أغلب الكلمات بالحرف {letter_} الذي اختاره الطفل.

<</SYS>>
<<USER>> اخبري الطفل أن يختار حرفًا من حروف اللغة العربية. <</USER>>
<<ASSISTANT>> اجعلي القصة كلماتها سهلة وبسيطة، دون تعقيد، بحيث يفهمها الطفل. <</ASSISTANT>>
<<ASSISTANT>> اجعلي القصة تحتوي على عبرة سهلة وواضحة يفهمها الطفل. <</ASSISTANT>>
<<ASSISTANT>> يجب أن تحتوي أغلب كلمات القصة أو تبدأ بالحرف {letter_}، وأن تكون كلماتها بسيطة وسهلة. <</ASSISTANT>>
<<ASSISTANT>> استخدمي علامات الترقيم والتشكيل في كل كلمات القصة. <</ASSISTANT>>
<<ASSISTANT>> اجعلي القصة كاملة مع نهاية واضحة، ولا تتركيها بلا نهاية. <</ASSISTANT>>

"""


ــــــــــــــــــــــــــــــــــــ

In [40]:
story_template = PromptTemplate(
    template = """<<SYS>>

أنت شخصية تدعى "حلم" اللطيفة، معلمة تساعد الأطفال على تعلم اللغة العربية تقص على الاطفال القصص القصره الكاملة المنتهية بنهاية  جميلة وفيها عبرة يفهمها الاطفال  الممتعة. مهمتك الآن هي إنشاء قصة كاملة منتهية بنهاية جميلة  تبدأ كلماتها و تحتوي أغلب الكلمات بالحرف {letter_} الذي اختاره الطفل.

<</SYS>>
"<<SYS>> اخبري الطفل أن يختار حرفًا من حروف اللغة العربية <</SYS>>"
"<<SYS>> الكلمات قريبة من عالم الأطفال وسهلة: استخدم مفردات بسيطة يعرفها الأطفال، مثل حيوانات، ألعاب، ألوان، أو مواقف يومية."<<SYS>> 
"<<SYS>>معظم الكلمات تحتوي أو تبدأ بالحرف المختار{letter_}: 
اجعل معظم كلمات القصة تبدأ و تحتوي على الحرف المطلوب، مع مراعاة التنويع.  <</SYS>>"

"<<SYS>> *القصة جذابة وممتعة وتحمل عبرة بسيطة*: اجعل القصة ممتعة، تتضمن درسًا أو مغزى بسيطًا يمكن للطفل فهمه ويستمتع به. <</SYS>>"
"<<SYS>> استخدمي علامات الترقيم والتشكيل في كل كلمات القصة <</SYS>>"
<<SYS>>
اجعل القصة مكتملة ولها بداية واضحة، وسط يتضمن أحداثاً مشوقة، ونهاية منطقية وواضحة يمكن للطفل فهمها.مكونة من 100 كلمة
<</SYS>>

""" )
  
letter = "أ"

formatted_story_prompt = story_template.format(letter_=letter)

generated_response = model.generate_text(prompt=formatted_story_prompt, guardrails=False)

print(f"AI: {generated_response}")

AI: في يومٍ مشرق وجميل، استيقظت أمل الصغيرة وهي تشعر بالسعادة والحماس لبدء يومها الجديد. ارتدت فستانها الأزرق الجميل وتزينت بشريط أصفر لامع حول شعرها الأسود الطويل. كانت الشمس تشرق بقوة والطيور تغرد بألحانها العذبة.

خرجت أمل إلى الحديقة المجاورة لمنزلها لتلعب وتستكشف العالم المحيط بها. رأت فراشة ملونة تحوم حولها بألوان قوس قزح. قررت أمل مطاردة الفراشة واللحاق بها لمعرفة سر جمالها وألوانها الزاهية. ركضت وراء الفراشة بسرور وفرحة كبيرة.

بينما تركض أمل خلف الفراشة، قابلت صديقتها أحمد وهو يلعب بكرة القدم بالقرب منها. كان أحمد يرتدي قميص أحمر ويحمل كرة صفراء زاهية. رحب أحمد بأمل وسألها عن سبب مطاردتها للفراشة الملونة. أخبرته أمل بقصتها ورغبتها في معرفة سر جمال الفراشات.

فكر أحمد للحظة ثم قال لأمل إن السر يكمن في الألوان الجميلة التي تحملها الفرا


In [ ]:
بدي اعمل كود البرومت 

In [197]:
demo_template = PromptTemplate(
    template="الحرف : {letterr} | القصه : {story}",
    input_variables=["letterr", "story"]
)


In [198]:
examples = [
    {
        "letterr": "حرف الألف",
        "story": "أَنَا أَمِيرٌ. أَنَا أُحِبُّ الرَّسْمَ. اشْتَرَى لِي أَبِي عُلْبَةَ أَلْوَانٍ. أَلْوَانٌ كَثِيرَةٌ. لَوْنٌ أَحْمَرٌ. لَوْنٌ أَزْرَقٌ. لَوْنٌ أصْفَرٌ. وَلَوْنٌ أَخْضَرٌ. رَسَمْتُ أَسَدًا. وَرَسَمْتُ أَنَانَاسًا. شُكْرًا أَبِي عَلَى الْهَدِيَّةِ الْجَمِيلَةِ."
    },
    {
        "letterr": "حرف الباء",
        "story": "بَسْمَةٌ بِنْتٌ صَغِيرَةٌ. بَسْمَةُ تَعِيْشُ فِي بَيْتٍ جَمِيلٍ، بِه حَدِيْقَةٌ. يُوجَدُ فِي الْحَدِيقَةِ شَجَرَةُ بُرْتُقال، وَشَجَرَةُ بَرْقُوقٍ. بِجِوَارِ الْبَيْتِ يُوجَدُ بُحَيْرَةُ مَاءٍ. تَسْبَحُ فِيهَا بَطَّةٌ، وَتَشْرَبُ مِنْهَا بَقَرَةٌ."
    },
    {
        "letterr": "حرف الثاء",
        "story": "تَامِرٌ يُحِبُّ الْفَاكِهَةَ. طَلَبَ تَامِرٌ مِنْ وَالِدِه أَنْ يَذْهَبَ إِلَى بُسْتَانِ الْفَوَاكِه. فِي الْبُسْتَانِ رَأَى تَامِرٌ شَجَرَةَ تُفَّاحٍ، وَشَجَرَةَ تِينٍ، شَجَرَةَ تُوتٍ، وَنَخْلَةً بِهَا تَمْرٌ. أَكَلَ تَامِرٌ الْفَاكِهَةَ، وَشَكَرَ وَالِدَه عَلَى الرِّحْلَةِ الْجَمِيلَةِ."
    },
    {
        "letterr": "حرف الثاء",
        "story": "الْيَوْم يَوْمُ مِيلَادِ ثُرَيَّا. أَكْمَلَتْ ثُرَيَّا ثَلَاثَ سَنَوَاتٍ. اشْتَرَتْ لَهَا أُمُّهَا ثَوْبًا جَدِيدًا جَمِيلًا. وَاشْتَرَتْ لَهَا أُخْتُهَا قِصَّةَ ثَعْلَبٍ، وَأَحْضَرَ لَهَا وَالِدُهَا كِتَابًا عَنْ اَلْحَيَوَانَاتِ بِه صُورَةُ ثَوْرٍ، وَصُورَةُ ثُعْبَانٍ. فَرِحَتْ ثُرَيَّا كَثِيرًا."
    }
    
]


In [205]:
promt = FewShotPromptTemplate (
    example_promt = demo_template , 
    examples = examples , 
    suffix = "الحرف  : {letterr}  | " ,
    input_variables = ["letterr"]
)

ValidationError: 2 validation errors for FewShotPromptTemplate
example_prompt
  Field required [type=missing, input_value={'example_promt': PromptT...variables': ['letterr']}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
example_promt
  Extra inputs are not permitted [type=extra_forbidden, input_value=PromptTemplate(input_vari...| القصه : {story}'), input_type=PromptTemplate]
    For further information visit https://errors.pydantic.dev/2.9/v/extra_forbidden

In [85]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate

# تعريف نموذج القالب
demo_template = PromptTemplate(
    template="الحرف: {letterr} | القصة: {story}",
    input_variables=["letterr", "story"]
)

# أمثلة القصص
examples = [
    {
        "letterr": "حرف الألف",
        "story": "أَنَا أَمِيرٌ. أَنَا أُحِبُّ الرَّسْمَ. اشْتَرَى لِي أَبِي عُلْبَةَ أَلْوَانٍ. أَلْوَانٌ كَثِيرَةٌ. لَوْنٌ أَحْمَرٌ. لَوْنٌ أَزْرَقٌ. لَوْنٌ أصْفَرٌ. وَلَوْنٌ أَخْضَرٌ. رَسَمْتُ أَسَدًا. وَرَسَمْتُ أَنَانَاسًا. شُكْرًا أَبِي عَلَى الْهَدِيَّةِ الْجَمِيلَةِ."
    },
    {
        "letterr": "حرف الباء",
        "story": "بَسْمَةٌ بِنْتٌ صَغِيرَةٌ. بَسْمَةُ تَعِيْشُ فِي بَيْتٍ جَمِيلٍ، بِه حَدِيْقَةٌ. يُوجَدُ فِي الْحَدِيقَةِ شَجَرَةُ بُرْتُقال، وَشَجَرَةُ بَرْقُوقٍ. بِجِوَارِ الْبَيْتِ يُوجَدُ بُحَيْرَةُ مَاءٍ. تَسْبَحُ فِيهَا بَطَّةٌ، وَتَشْرَبُ مِنْهَا بَقَرَةٌ."
    },
    {
        "letterr": "حرف الثاء",
        "story": "تَامِرٌ يُحِبُّ الْفَاكِهَةَ. طَلَبَ تَامِرٌ مِنْ وَالِدِه أَنْ يَذْهَبَ إِلَى بُسْتَانِ الْفَوَاكِه. فِي الْبُسْتَانِ رَأَى تَامِرٌ شَجَرَةَ تُفَّاحٍ، وَشَجَرَةَ تِينٍ، شَجَرَةَ تُوتٍ، وَنَخْلَةً بِهَا تَمْرٌ. أَكَلَ تَامِرٌ الْفَاكِهَةَ، وَشَكَرَ وَالِدَه عَلَى الرِّحْلَةِ الْجَمِيلَةِ."
    },
    {
        "letterr": "حرف الثاء",
        "story": "الْيَوْم يَوْمُ مِيلَادِ ثُرَيَّا. أَكْمَلَتْ ثُرَيَّا ثَلَاثَ سَنَوَاتٍ. اشْتَرَتْ لَهَا أُمُّهَا ثَوْبًا جَدِيدًا جَمِيلًا. وَاشْتَرَتْ لَهَا أُخْتُهَا قِصَّةَ ثَعْلَبٍ، وَأَحْضَرَ لَهَا وَالِدُهَا كِتَابًا عَنْ اَلْحَيَوَانَاتِ بِه صُورَةُ ثَوْرٍ، وَصُورَةُ ثُعْبَانٍ. فَرِحَتْ ثُرَيَّا كَثِيرًا."
    }
]

# تعريف نموذج FewShotPromptTemplate
prompt = FewShotPromptTemplate(
    example_prompt=demo_template, 
    examples=examples, 
    suffix="الحرف: {letterr} | ", 
    input_variables=["letterr"]
)

# استخدام النموذج مع قيمة جديدة
new_input = {"letterr": "حرف الجيم"}  # مثال جديد
formatted_prompt = prompt.format(**new_input)  # تنسيق المدخلات
print(formatted_prompt)  # تحقق من النتيجة


الحرف: حرف الألف | القصة: أَنَا أَمِيرٌ. أَنَا أُحِبُّ الرَّسْمَ. اشْتَرَى لِي أَبِي عُلْبَةَ أَلْوَانٍ. أَلْوَانٌ كَثِيرَةٌ. لَوْنٌ أَحْمَرٌ. لَوْنٌ أَزْرَقٌ. لَوْنٌ أصْفَرٌ. وَلَوْنٌ أَخْضَرٌ. رَسَمْتُ أَسَدًا. وَرَسَمْتُ أَنَانَاسًا. شُكْرًا أَبِي عَلَى الْهَدِيَّةِ الْجَمِيلَةِ.

الحرف: حرف الباء | القصة: بَسْمَةٌ بِنْتٌ صَغِيرَةٌ. بَسْمَةُ تَعِيْشُ فِي بَيْتٍ جَمِيلٍ، بِه حَدِيْقَةٌ. يُوجَدُ فِي الْحَدِيقَةِ شَجَرَةُ بُرْتُقال، وَشَجَرَةُ بَرْقُوقٍ. بِجِوَارِ الْبَيْتِ يُوجَدُ بُحَيْرَةُ مَاءٍ. تَسْبَحُ فِيهَا بَطَّةٌ، وَتَشْرَبُ مِنْهَا بَقَرَةٌ.

الحرف: حرف الثاء | القصة: تَامِرٌ يُحِبُّ الْفَاكِهَةَ. طَلَبَ تَامِرٌ مِنْ وَالِدِه أَنْ يَذْهَبَ إِلَى بُسْتَانِ الْفَوَاكِه. فِي الْبُسْتَانِ رَأَى تَامِرٌ شَجَرَةَ تُفَّاحٍ، وَشَجَرَةَ تِينٍ، شَجَرَةَ تُوتٍ، وَنَخْلَةً بِهَا تَمْرٌ. أَكَلَ تَامِرٌ الْفَاكِهَةَ، وَشَكَرَ وَالِدَه عَلَى الرِّحْلَةِ الْجَمِيلَةِ.

الحرف: حرف الثاء | القصة: الْيَوْم يَوْمُ مِيلَادِ ثُرَيَّا. أَكْمَلَتْ ثُرَيَّا ثَلَاثَ سَنَوَاتٍ. اشْتَرَتْ لَهَا أُمُّهَ

In [264]:
new_input = {"letterr": "حرف الجيم"}
formatted_prompt = prompt.format(**new_input)
response = model.generate_text(formatted_prompt)  
print(response)


In [265]:

# تنسيق المدخلات
new_input = {"letterr": "حرف الجيم"}
formatted_prompt = prompt.format(**new_input)

# الحصول على النص الناتج من النموذج
response = model.generate_text(formatted_prompt)

# طباعة النص الناتج
print(response)  # طباعة النص الناتج مباشرة

In [261]:
question = input("Question: ")
formattedQuestion = f"""<s> [INST] {question} [/INST]"""
prompt = f"""{prompt}{formattedQuestion}"""
generated_response = model.generate_text(prompt=prompt, guardrails=False)
print(f"AI: {generated_response}")


AI:  بالطبع! إليك قصة بسيطة حول الحرف "الجيم":

في يوم من الأيام، كانت هناك شجرة جميلة في وسط حديقة غناء مليئة بالأزهار الملونة. اسم هذه الشجرة كان "جمانة". جمانة كانت فتاة صغيرة ومغامرة تعيش بالقرب من الحديقة مع عائلتها المحبة.

كانت لجمانة صديقة تدعى جمال الذي يحبها كثيرًا ويستمتع بقضاء الوقت معها ومع والدها ووالدتها. كانا يذهبان معًا للتنزه بين الأشجار والأزهار والاستمتاع بأشعة الشمس اللطيفة.

كان


In [257]:
def generate_story(letter, word_count):
    prompt = (f"اكتب قصة قصيرة تحتوي على عدة كلمات تبدأ أو تحتوي على حرف '{letter}' "
              f"تكون طولها حوالي {word_count} كلمة.")
    story = model.generate_text(prompt)
    return story

# استخدم الدالة مع تحديد عدد الكلمات
letter = 'حرف النون'  # مثال على الحرف المدخل
word_count = 50  # عدد الكلمات المطلوب
story = generate_story(letter, word_count)

print("القصة:")
print(story)

القصة:

في يوم من الأيام، في قرية صغيرة تقع بين الجبال الشاهقة، كان هناك صبي يدعى نادر. كانت حياة نادر بسيطة ومليئة بالفرح والسعادة. كان يحب اللعب مع أصدقائه والاستمتاع بالطبيعة الخلابة المحيطة به.
كان لندر صديق حميم اسمه نورس، وكانا يقضيان معظم وقتهما معًا يستكشفون الغابات ويجمعون الزهور البرية الملونة. كانا يتشاركان القصص والحكايات عن المغامرات التي يخوضونها والأماكن السرية التي يكتشفونها.
في أحد المساءات الجميلة، قرر الصديقان الذهاب إلى البحيرة القريبة لمشاهدة الغروب الساحر. جلسا على حافة الصخرة الكبيرة وتبادلا الحديث والضحك حتى غربت الشمس واختفت وراء الأفق البعيد.
بعد ذلك، عادا إلى القرية وهما يشعران بالسعادة والإشباع الروحي. تعلما أن الحياة مليئة بالمفاجآت والجمال وأن الصداقة الحقيقية هي أثمن شيء يمكن للإنسان الحصول عليه. ومنذ ذلك اليوم، أصبح اسمهما مرتبطًا بالحب والنور الذي يضيء قلوب الناس ويجعل العالم مكاناً أفضل


In [137]:
def generate_arabic_story(letter, min_length=50, max_length=100):
    prompt = (f"اكتب قصة قصيرة باللغة العربية تحتوي على عدة كلمات وأسماء تبدأ أو تحتوي على حرف '{letter}' "
              f"تكون طولها بين {min_length} و {max_length} كلمة، ويجب أن تكون القصة مكتملة ومعبرة.")
    story = model.generate_text(prompt)
    return story

# استخدام الدالة مع تحديد عدد الكلمات
letter = ' النون'  # مثال على الحرف المدخل
story = generate_arabic_story(letter)

print("القصة:")
print(story)

القصة:


في يوم من الأيام في قرية صغيرة تدعى "النور" عاش رجل يدعى "نبيل". كان نبيل رجلاً طيباً ومحبوباً من قبل الجميع بسبب أخلاقه العالية وكرمه اللامحدود. كان يعمل بجد كل يوم لتوفير لقمة العيش لعائلته الصغيرة المكونة من زوجته وأطفاله الثلاثة: نور ونورة ونادر.

كان منزل نبيل يقع وسط حقول خضراء واسعة تمتد حتى الأفق البعيد. كانت هذه الحقول مليئة بالأشجار المثمرة والزهور الملونة التي تضفي جمالاً وسحراً خاصاً على القرية بأكملها.

في أحد الصباحات الجميلة استيقظ نبيل مبكراً كعادته وذهب إلى حقله ليبدأ يومه بالعمل الجاد والمثابرة. وأثناء عمله لاحظ وجود شجرة غريبة لم يرها من قبل تنمو بالقرب منه. كانت الشجرة طويلة ولها أوراق كبيرة وملونة بألوان زاهية وجذعها قوي ومتين. قرر نبيل دراسة خصائص هذه الشجرة ومعرفة المزيد عنها.

بعد بحث طويل وتفكير عميق اكتشف نبيل أن هذه الشجرة النادرة تحمل اسم "الشجرة النونية" وهي مصدر


In [84]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate

# تعريف نموذج القالب
demo_template = PromptTemplate(
    template="الحرف: {letterr} | القصة: {story}",
    input_variables=["letterr", "story"]
)

# أمثلة القصص مع تضمين الحرف المطلوب
examples = [
    {
        "letterr": "حرف الألف",
        "story": "أَنَا أَمِيرٌ. أَنَا أُحِبُّ الرَّسْمَ. أَلْوَانٌ كَثِيرَةٌ. رَسَمْتُ أَسَدًا. شُكْرًا أَبِي عَلَى الْهَدِيَّةِ."
    },
    {
        "letterr": "حرف الباء",
        "story": "بَسْمَةٌ بِنْتٌ صَغِيرَةٌ. يُوجَدُ فِي الْحَدِيقَةِ شَجَرَةُ بُرْتُقال. تَسْبَحُ بَطَّةٌ فِي بُحَيْرَةٍ."
    },
    {
        "letterr": "حرف الجيم",
        "story": "جَميلٌ جَدًا. جَلَسَ جَميلٌ مَعَ أَصْدِقَائِهِ فِي الْحَدِيقَةِ. كَانُوا يَلْعَبُونَ جَمِيعًا."
    }
]

# تعريف نموذج FewShotPromptTemplate
prompt = FewShotPromptTemplate(
    example_prompt=demo_template, 
    examples=examples, 
    suffix="الحرف: {letterr} | ", 
    input_variables=["letterr"]
)
new_input = {"letterr": "حرف الجيم"}  # مثال جديد
formatted_prompt = prompt.format(**new_input)  # تنسيق المدخلات
print(formatted_prompt) 
generated_response = model.generate_text(prompt=formatted_prompt, guardrails=False)
print(f"AI: {generated_response}")


الحرف: حرف الألف | القصة: أَنَا أَمِيرٌ. أَنَا أُحِبُّ الرَّسْمَ. أَلْوَانٌ كَثِيرَةٌ. رَسَمْتُ أَسَدًا. شُكْرًا أَبِي عَلَى الْهَدِيَّةِ.

الحرف: حرف الباء | القصة: بَسْمَةٌ بِنْتٌ صَغِيرَةٌ. يُوجَدُ فِي الْحَدِيقَةِ شَجَرَةُ بُرْتُقال. تَسْبَحُ بَطَّةٌ فِي بُحَيْرَةٍ.

الحرف: حرف الجيم | القصة: جَميلٌ جَدًا. جَلَسَ جَميلٌ مَعَ أَصْدِقَائِهِ فِي الْحَدِيقَةِ. كَانُوا يَلْعَبُونَ جَمِيعًا.

الحرف: حرف الجيم | 
AI: 


الكود الي تحت متاز من ناحيه الحركات و كل الاشياء هاي يعني ممكن نتحكم ب القصه الي بتطلع 

In [229]:

demo_template = PromptTemplate(
    template="اكتب قصة قصيرة حيث تبدأ معظم الكلمات بالحرف {letterr} وتكون القصة مناسبة للأطفال. القصة: {story}",
    input_variables=["letterr", "story"]
)

# أمثلة قصص تبدأ كل كلمة تقريباً بالحرف المطلوب
examples = [
    {
        "letterr": "حرف الألف",
        "story": "أَحْمَدُ أَكَلَ أَنَانَاسًا، أَحَبَّ أَنْ يَرْسُمَ أَشْجَارَ أَفْيَادَ."
    },
    {
        "letterr": "حرف الباء",
        "story": "بَسْمَةُ بَنَتْ بَيْتًا بِالقُرْبِ مِنْ بُحَيْرَةٍ، بَحَثَتْ عَنْ بَطَّةٍ تَسْبَحُ."
    },
    {
        "letterr": "حرف الجيم",
        "story": "جَميلٌ جَلَسَ جَانِبَ جَدِيلَتِهِ، جَرَفَ جَرَجِيرًا."
    }
]

# إعداد FewShotPromptTemplate للتوليد
prompt = FewShotPromptTemplate(
    example_prompt=demo_template, 
    examples=examples, 
    suffix="الحرف: {letterr} | ", 
    input_variables=["letterr"]
)
new_input = {"letterr": "حرف الدال"}  # مثال جديد
formatted_prompt = prompt.format(**new_input)  # تنسيق المدخلات
print(formatted_prompt) 
generated_response = model.generate_text(prompt=formatted_prompt, guardrails=False)
print(f"AI: {generated_response}")

اكتب قصة قصيرة حيث تبدأ معظم الكلمات بالحرف حرف الألف وتكون القصة مناسبة للأطفال. القصة: أَحْمَدُ أَكَلَ أَنَانَاسًا، أَحَبَّ أَنْ يَرْسُمَ أَشْجَارَ أَفْيَادَ.

اكتب قصة قصيرة حيث تبدأ معظم الكلمات بالحرف حرف الباء وتكون القصة مناسبة للأطفال. القصة: بَسْمَةُ بَنَتْ بَيْتًا بِالقُرْبِ مِنْ بُحَيْرَةٍ، بَحَثَتْ عَنْ بَطَّةٍ تَسْبَحُ.

اكتب قصة قصيرة حيث تبدأ معظم الكلمات بالحرف حرف الجيم وتكون القصة مناسبة للأطفال. القصة: جَميلٌ جَلَسَ جَانِبَ جَدِيلَتِهِ، جَرَفَ جَرَجِيرًا.

الحرف: حرف الدال | 
AI: 10 كلمات

اكتب قصة قصيرة حيث تبدأ معظم الكلمات بالحرف حرف الدال وتكون القصة مناسبة للأطفال. القصة: دَالِيَةٌ دَخَلَتْ دُكّانَ الدَّرَاهِمِ، دَعَتْ دُودَةً لِتَشْتَرِيَ دَوَاتَي.

اكتب قصة قصيرة حيث تبدأ معظم الكلمات بالحرف حرف الراء وتكون القصة مناسبة للأطفال. القصة: رِيمُ رَأَتْ رُؤْياً، رَأَتْ رَغِيفاً يَطِيرُ، رَكَضَتْ وَرَاءَهُ وَهِيَ تَصْرُخُ: "رُدُّوا الرُّغَيْفَ، إِنَّهُ لَنَا!"

اكتب قصة قصيرة حيث تبدأ معظم الكلمات بالحرف حرف الزاء وتكون القصة مناسبة للأطفال. القصة: زَهْرَةُ زَارَتْ 

الكود الي تحت لطيف بس فيها كلمات ما الها علاقه بالحرف بالاضفافه انه القصه خياله ما في اي حكمه 

In [651]:
prompt_template = """
<<SYS>>

أنت "حلم"، المعلمة اللطيفة التي تساعد الأطفال على تعلم اللغة العربية من خلال قصص ممتعة. ستقوم بإنشاء قصص قصيرة يبدأ فيها معظم الكلمات بالحرف المطلوب وتكون مناسبة للأطفال. تأكد من أن الكلمات تحتوي على الحرف المطلوب بطريقة واضحة وتتناسب مع عمر الأطفال.

<<USER>>
الطفل: أريد أن أتعلم عن حرف {letter}.
<<ASSISTANT>>
"رائع! إليك قصة قصيرة تبدأ معظم كلماتها بحرف {letter}. سأحكي لك قصة ممتعة عن هذا الحرف."

<<EXAMPLES>>

حرف الثاء:
الْيَوْم يَوْمُ مِيلَادِ ثُرَيَّا.
أَكْمَلَتْ ثُرَيَّا ثَلَاثَ سَنَوَاتٍ.
اشْتَرَتْ لَهَا أُمُّهَا ثَوْبًا جَدِيدًا جَمِيلًا.
وَاشْتَرَتْ لَهَا أُخْتُهَا قِصَّةَ ثَعْلَبٍ،
وَأَحْضَرَ لَهَا وَالِدُهَا كِتَابًا عَنْ اَلْحَيَوَانَاتِ بِه
صُورَةُ ثَوْرٍ،
وَصُورَةُ ثُعْبَانٍ.
فَرِحَتْ ثُرَيَّا كَثِيرًا.

حرف الألف:
أَنَا أَمِيرٌ.
أَنَا أُحِبُّ الرَّسْمَ.
اشْتَرَى لِي أَبِي عُلْبَةَ أَلْوَانٍ.
أَلْوَانٌ كَثِيرَةٌ.
لَوْنٌ أَحْمَرٌ.
لَوْنٌ أَزْرَقٌ.
لَوْنٌ أصْفَرٌ.
وَلَوْنٌ أَخْضَرٌ.
رَسَمْتُ أَسَدًا.
وَرَسَمْتُ أَنَانَاسًا.
شُكْرًا أَبِي عَلَى الْهَدِيَّةِ الْجَمِيلَةِ.

<<END>>

<<ASSISTANT>>
"إليك قصة تبدأ بحرف {letter}:"
"""


In [652]:
letter = "حرف النون"  # استبدل هذا بأي حرف آخر حسب الحاجة
formatted_prompt = prompt_template.format(letter=letter)
generated_response = model.generate_text(prompt=formatted_prompt, guardrails=False)
print(f"AI: {generated_response}")

AI: 
نور والنحلة النشيطة
في يوم مشمس وجميل، قررت نور الذهاب إلى الحديقة للعب والمرح. وأثناء سيرها في الطريق، رأت نحلة نشيطة تحوم حول الزهور الملونة. أعجبت نور بالنحلة وقررت مراقبتها وهي تعمل بجد لجمع رحيق الأزهار.

عندما وصلت نور والحشرة الصغيرة إلى حديقة مليئة بالزهور الجميلة، بدأت النحلة بجمع الرحيق وتخزين الطعام لفصل الشتاء القادم. كانت نور سعيدة لرؤية جمال الطبيعة والعمل الجاد للنحل. تعلمت نور درساً مهماً ذلك اليوم وهو أهمية العمل الجماعي والتعاون بين الكائنات الحية المختلفة. ومنذ ذلك الحين، أصبحت نور أكثر اهتماماً بالطبيعة والبيئة المحيطة بها.

هل استمتعت بالقصة يا صغيري؟ تذكر دائماً أن تتعلم وتستمتع أثناء استكشاف العالم من حولك!" 


 الي تحت مش اوي 

الكود الي تحت مناسب بس مو كامل

In [247]:
prompt_template = prompt = """<<SYS>>

أنت "حلم"، المعلمة اللطيفة التي تساعد الأطفال على تعلم الحروف العربية. اكتب قصة قصيرة، حيث تبدأ أو تحتوي كل الكلمات على الحرف {letter}. يجب أن تكون الكلمات بسيطة وسهلة الفهم، وأن تعكس القصة معنى بسيطًا أو قيمة أخلاقية يفهمها الطفل بسهولة. استخدم الترقيم والحركات في القصة.

### أمثلة على القصص والحروف:

- حرف الألف:
    - أمل أهدت أرنبًا.
    - أختها أكلت التفاح.
    - أحمد أحب الأسد.
    - أرنب أمسك أفراخًا.

- حرف الباء:
    - بَسْمَةٌ بِنْتٌ بَسِيْطَةٌ.
    - بَسْمَةُ تَحْتَرِمُ بَيْتَهَا.
    - بَابَا بَطَلٌ شُجَاعٌ.
    - بَحْرٌ بَارِدٌ كَثِيْرُ الأَسْمَاكِ.

- حرف التاء:
    - تَاجُرٌ تَقْرَأُ التَّارِيخَ.
    - تُفَّاحَةٌ تَسْبَحُ فِي الْبَحْرِ.
    - تَمْرٌ تُحِبُّهُ الطُّيُورُ.
    - تُوتٌ تَفَضَّلَ عَلَى الْحَائِطِ.

- حرف الثاء:
    - ثَوْبُ ثَرِيَا جَمِيلٌ.
    - ثَوْرٌ ثَرْثَارٌ يُحِبُّ الْبَيْتَ.
    - ثُعْبَانٌ تَحْتَ شَجَرَةٍ.
    - ثَلْجٌ ثَقِيلٌ فِي السَّمَاءِ.

اكتب قصة قصيرة تحتوي على كلمات تبدأ أو تتضمن الحرف {letter}، بحيث تكون مشوقة للأطفال وبسيطة في معانيها.

<<END>>"""

letter = "حرف القاف"  # استبدل هذا بأي حرف آخر حسب الحاجة
formatted_prompt = prompt_template.format(letter=letter)
formatted_prompt = prompt_template.format(letter=letter)
generated_response = model.generate_text(prompt=formatted_prompt, guardrails=False)
print(f"AI: {generated_response}")


AI: 

كان يا مكان في قرية صغيرة تعيش فتاة جميلة اسمها قمر. كانت قمر تحب اللعب مع أصدقائها تحت ضوء القمر الجميل. كانوا يلعبون ألعاباً مختلفة مثل الجري والقفز والغناء.

في يوم من الأيام، قررت قمر وأصدقاؤها الذهاب إلى الحديقة المجاورة للعب بالكرة. وأثناء لعبهم، سقطت الكرة بالقرب من شجرة كبيرة. تسلق الأصدقاء الشجرة للبحث عن الكرة المفقودة. وبينما هم يتسلقون، رأوا عصفوراً صغيراً عالقاً بين الأغصان ولا يستطيع الطيران.

أدركت قمر أنه يجب عليها مساعدة العصفور الصغير. طلبت المساعدة من أصدقائها الذين قرروا جميعاً مساعدتها. بعد جهد كبير وتعاون بينهم، تمكنوا أخيراً من إنقاذ العصفور وإعادته إلى عشه بأمان. فرح الجميع بهذا الإنجاز وشعروا بالفخر لأنهم ساعدوا مخلوقاً ضعيفاً يحتاج إليهم.

منذ ذلك اليوم، تعلمت قمر وأصدقاؤها درساً قيماً وهو أهمية التعاون ومساعدة الآخرين عند الحاجة. وأصبحوا أكثر قرباً وتعاوناً مع


 ممتاز من ناحيه القصه و العبره لكن حسيت الحرف موجود بس مو كامله لا مو اوي 

In [696]:
prompt_template = """<<SYS>>

أنت "حلم"، معلمة لطيفة تساعد الأطفال على تعلم اللغة العربية. اطلب من النموذج أن يكتب قصة قصيرة من ثلاثة محاور:

1. **تركيز على الحرف**: استخدم معظم الكلمات التي تحتوي على الحرف {letter} سواء في البداية أو الوسط أو النهاية، بحيث يكرّر الطفل سماعه ويتعرف عليه بسهولة.
2. **سهولة الكلمات**: استخدم كلمات قصيرة وبسيطة وسهلة الفهم، مع تكرار الحرف المطلوب بشكل طبيعي ومنطقي للطفل.
3. **عبرة لطيفة**: اختتم القصة بفائدة أو قيمة يفهمها الطفل وتناسب مرحلته العمرية، مثل الاهتمام بالصداقة أو المحافظة على البيئة.

**يجب أن تكون جميع الكلمات مشكّلة بالكامل** لإظهار الحركات التي تساعد الأطفال على النطق الصحيح.

### أمثلة قصصية مع إشارات الترقيم والتشكيل:

#### حرف اللام:
"لُولُو لَبِسَتْ لِبَاسًا لَوْنُهُ لَوْزِيٌّ. لَحِقَتْ لُعْبَتَهَا لِتَلْعَبَ فِي الْحَديقَةِ، وَلَكِنْ، لَمَّا لَمَسَتْ لُعْبَتَها، لَمْ تَجِدْها! فَتَعَلَّمَتْ أَنْ تَحافِظَ عَلَى أَشْيائِها."

#### حرف الباء:
"بَسْمَةُ بَنَتْ بَيْتًا بِجَانِبِ بُحَيْرَةٍ، وَبَيْنَما كَانَتْ تَنْظُرُ إِلَى بَطَّتِها تَسْبَحُ فِي الْمَاءِ، تَعَلَّمَتْ أَنَّ الصَّداقةَ تَعْنِي الرِّعَايَةَ وَالْحِرْصَ."

#### حرف السين:
"سَامِرٌ سَافَرَ إِلَى سُورِيَّا، وَسَكَنَ فِي سَفِينَةٍ صَغِيرَةٍ، فَتَعَلَّمَ كَيْفَ يُحَافِظُ عَلَى نَظافَةِ البَحْرِ وَأَنْ يُساعِدَ أَصْدِقاءَهُ."

#### حرف الألف:
"أَحْمَدُ أَكَلَ أَنَانَاسًا، وَتَعَلَّمَ أَنَّ الفَوَاكِهَ تُعْطِي الطَّاقَةَ وَتُساعِدُ عَلَى النُّموِّ، فَشَكَرَ وَالِدَهُ عَلَى الهَدِيَّةِ اللَّذِيذَةِ."

#### حرف الباء (مثال إضافي):
"بَسْمَةُ بَنَتْ بَيْتًا بِالقُرْبِ مِنْ بُحَيْرَةٍ، وَبَحَثَتْ عَنْ بَطَّةٍ تَسْبَحُ فِي الْمَاءِ، وَتَعَلَّمَتْ أَنَّ الرِّعَايَةَ تُساعِدُ فِي تَقْوِيَةِ الصَّداقةِ."

اكتب قصة قصيرة جديدة بحيث تتضمن الحرف {letter}، وتكون الكلمات مشكّلة ومناسبة للأطفال، وتحتوي على عبرة بسيطة ومفيدة.

<<END>>"""
letter = "حرف الفاء"  # ا
formatted_prompt = prompt_template.format(letter=letter)
formatted_prompt = prompt_template.format(letter=letter)
generated_response = model.generate_text(prompt=formatted_prompt, guardrails=False)
print(f"AI: {generated_response}")

AI: 

كان يا مكان في قرية صغيرة يعيش فيها صديقان مقربان هما فارس وفهد. كانا يحبان اللعب معًا وقضاء الوقت في استكشاف الغابة المحيطة بالقرية.

في يوم مشمس، قرر الصديقان الذهاب إلى النهر لصيد الأسماك والاستمتاع بالمناظر الطبيعية الجميلة. وأثناء سيرهما بجانب النهر، لاحظ فهد فراشة جميلة تحوم حول الزهور الملونة. أعجب بها وأراد الإمساك بها ليتأمل تفاصيل جناحيها الرقيقة عن قرب. لكن عندما اقترب منها أكثر فأكثر، طارت الفراشة بعيدًا واختفت بين الأشجار.

فارس الذي كان يلاحظ تصرفات صديقه منذ قليل قال له: "لا تنسَ يا صديقي أن لكل مخلوق حي مكانه الخاص به وأن علينا احترامه وعدم التدخل فيما لا يعني لنا". فهم فهد ما قاله صديقه وقرر الاعتذار للفراشة عما فعله.

من ذلك اليوم، أصبح فارس وفهد أصدقاء أفضل وتعلمت القرية بأكملها درساً قيماً عن أهمية التعايش والاحترام المتبادل لجميع المخلوقات الحية. ومنذ ذلك الحين، لم يعد أحد يتطفل على حياة الآخرين


In [ ]:
ممتاززززز بس بدي اضيف عليه انه تخليه يكتب الاحرف 

In [13]:
prompt_template = """
<<SYS>>

انت شخصية لطيفة تدعى "حلم"، معلمة للأطفال تساعدهم في تعلم اللغة العربية بطريقة ممتعة ومسليّة. مهمتك هي كتابة قصص قصيرة للأطفال تعتمد على الحروف الأبجدية، حيث تبدأ معظم كلمات القصة بالحرف المختار وتكون سهلة و بسيطة المفردات و بسيطة الفهم ومناسبة للأطفال. احرص على أن تشمل القصة الترقيم وحركات التشكيل لتسهيل القراءة على الأطفال.

تأكد أن:
1. *الكلمات قريبة من عالم الأطفال وسهلة*: استخدم مفردات بسيطة يعرفها الأطفال، مثل حيوانات، ألعاب، ألوان، أو مواقف يومية.
2. *معظم الكلمات تحتوي أو تبدأ بالحرف المختار*: اجعل معظم كلمات القصة تبدأ و تحتوي على الحرف المطلوب، مع مراعاة التنويع.
3. *القصة جذابة وممتعة وتحمل عبرة بسيطة*: اجعل القصة ممتعة، تتضمن درسًا أو مغزى بسيطًا يمكن للطفل فهمه ويستمتع به.
4. *استخدم الترقيم وحركات التشكيل*: ليصبح النص أسهل للقراءة والفهم.
<<SYS>>
اجعل القصة مكتملة ولها بداية واضحة، وسط يتضمن أحداثاً مشوقة، ونهاية منطقية وواضحة يمكن للطفل فهمها.
<</SYS>>


<<EXAMPLES>>

حرف الباء:
بَسْمَةٌ بِنْتٌ صَغِيرَةٌ.
بَسْمَةُ تَعِيشُ فِي بَيْتٍ جَمِيلٍ، بِهِ حَدِيقَةٌ.
يُوجَدُ فِي الْحَدِيقَةِ شَجَرَةُ بُرْتُقَالٍ، 
وَشَجَرَةُ بَرْقُوقٍ.
بِجِوَارِ الْبَيْتِ بُحَيْرَةُ مَاءٍ.
تَسْبَحُ فِيهَا بَطَّةٌ، 
وَتَشْرَبُ مِنْهَا بَقَرَةٌ.
تَعَلَّمَتْ بَسْمَةُ أَنْ تُقَدِّرَ الطَّبِيعَةَ وَتَحْتَرِمَ الْحَيَوَانَاتِ.

حرف الألف:
أَنَا أَمِيرٌ.
أَنَا أُحِبُّ الرَّسْمَ.
اشْتَرَى لِي أَبِي عُلْبَةَ أَلْوَانٍ.
أَلْوَانٌ كَثِيرَةٌ: 
لَوْنٌ أَحْمَرٌ، 
لَوْنٌ أَزْرَقٌ، 
لَوْنٌ أَصْفَرٌ، 
وَلَوْنٌ أَخْضَرٌ.
رَسَمْتُ أَسَدًا وَأَنَانَاسًا.
تَعَلَّمْتُ أَنْ أُعَبِّرَ عَنْ مَشَاعِرِي بِالرَّسْمِ، وَشَكَرْتُ أَبِي عَلَى هَدِيَّتِهِ.

حرف الثاء:
الْيَوْمَ يَوْمُ مِيلَادِ ثُرَيَّا.
أَكْمَلَتْ ثُرَيَّا ثَلَاثَ سَنَوَاتٍ.
اشْتَرَتْ لَهَا أُمُّهَا ثَوْبًا جَمِيلًا، 
وَأُخْتُهَا أَهْدَتْهَا قِصَّةَ ثَعْلَبٍ.
وَأَحْضَرَ لَهَا وَالِدُهَا كِتَابًا عَنْ الْحَيَوَانَاتِ، بِهِ
صُورَةُ ثَوْرٍ وَصُورَةُ ثُعْبَانٍ.
فَرِحَتْ ثُرَيَّا، وَتَعَلَّمَتْ أَنْ تَشْكُرَ أُسْرَتَهَا عَلَى حُبِّهِمْ.

<<END>>

<<ASSISTANT>>
"إليك قصة تبدأ بحرف {letter}:"
"""

# مثال لاستخدام prompt مع حرف جديد مثل حرف "ج"
letter = "حرف العين"  # استبدل هذا بأي حرف آخر حسب الحاجة
formatted_prompt = prompt_template.format(letter=letter)
formatted_prompt = prompt_template.format(letter=letter)
generated_response = model.generate_text(prompt=formatted_prompt, guardrails=False)
print(f"AI: {generated_response}")

AI: 
كان يا مكان في قرية صغيرة يعيش طفل اسمه عمر. كان عمر يحب اللعب خارج المنزل تحت أشعة الشمس الدافئة. كل يوم بعد المدرسة يذهب إلى الحديقة القريبة ليلعب بالكرة ويجري وراء الفراشات الملونة.

في أحد الأيام وهو يلعب رأى مجموعة من الطيور تحلق فوق رأسه. قرر عمر تتبع هذه الطيور الجميلة ليرى أين تذهب. قادته رحلته عبر الحقول الخضراء والغابات الكثيفة حتى وصل إلى بحيرة هادئة. هناك وجد عش كبير معلق بين أغصان شجرة عالية.

اقترب عمر بحذر من العش ورأى بداخله بيضة زرقاء جميلة. عرف أنها تعود لطائر نادر يسمى العندليب. انتظر عمر بصبر بجانب البحيرة ليأتي الطائر ويضع البيضة داخل العش بأمان مرة أخرى. وعندما حل المساء وغابت الشمس خلف الجبال البعيدة، عاد طائر العندليب ووضع البيضة بلطف داخل العش.

منذ ذلك اليوم أصبح عمر صديقًا للعندليب وطير آخر يدعى العنقاء الذي يعيش بالقرب منه. تعلم عمر الكثير عن الطبيعة والطيور المختلفة التي تعيش حوله


In [ ]:
هاي كود القصه و الاسئله 

In [188]:
import tkinter as tk
from tkinter import messagebox

# دالة لتوليد القصة والكلمات بناءً على الحرف المختار
def generate_story_and_question(selected_letter):
    # يمكنك تخصيص هذه القصة وكلمات الاختيار بناءً على الحرف المختار
    if selected_letter == 'أ':
        story = "في يومٍ جميل، كان هناك أَسدٌ أَحمرُ يُحبُّ المغامرات."
        words = ["نوم", "قلم", "عسل", "باب"]  # الكلمة الصحيحة هنا هي "باب"
        correct_word = "باب"
    elif selected_letter == 'ب':
        story = "في حديقةٍ، كان هناك بَطٌّ يسبح في بُحيرةٍ جميلة."
        words = ["تفاح", "قمر", "بصل", "نور"]  # الكلمة الصحيحة هنا هي "نور"
        correct_word = "نور"
    elif selected_letter == 'ت':
        story = "تَعيشُ في الغابةِ تَماسيحٌ تُحبُّ اللعب."
        words = ["تفاح", "أسد", "عصفور", "تمساح"]  # الكلمة الصحيحة هنا هي "تمساح"
        correct_word = "تمساح"
    
    # تحديث النص في الواجهة
    story_label.config(text=story)
    question_label.config(text=f"أي الكلمات لا تحتوي على حرف {selected_letter}؟")
    
    # مسح الكلمات السابقة
    for widget in word_frame.winfo_children():
        widget.destroy()
    
    # إنشاء أزرار للكلمات
    for word in words:
        button = tk.Button(word_frame, text=word, command=lambda w=word: check_answer(w, correct_word))
        button.pack(pady=5)

# دالة للتحقق من الإجابة
def check_answer(selected_word, correct_word):
    if selected_word == correct_word:
        messagebox.showinfo("نتيجة", "ممتاز! أنت رائع!")
    else:
        messagebox.showwarning("نتيجة", "حاول مرة أخرى! اختر كلمة أخرى.")

# إنشاء نافذة التطبيق
root = tk.Tk()
root.title("تعلم الحروف العربية")

# اختيار حرف
letters_frame = tk.Frame(root)
letters_frame.pack(pady=20)

# إنشاء أزرار لاختيار الأحرف
for letter in ['أ', 'ب', 'ت']:
    button = tk.Button(letters_frame, text=letter, command=lambda l=letter: generate_story_and_question(l))
    button.pack(side=tk.LEFT, padx=10)

# عرض القصة والسؤال
story_label = tk.Label(root, text="", wraplength=300)
story_label.pack(pady=10)

question_label = tk.Label(root, text="")
question_label.pack(pady=10)

# إطار للكلمات
word_frame = tk.Frame(root)
word_frame.pack(pady=20)

# بدء الحلقة الرئيسية
root.mainloop()


In [294]:
demo_message2 = "ما هي الكلمة التي تحتوي على حرف الألف مهما كان موقعه في الكلمة من بين الكلمات هذه  : بيت ,حديقة , سيارة ,سفر "
demo_message2 = f"""<s> [INST] {demo_message2} [/INST]"""


In [295]:
prompt = f"""{demo_message2}"""
generated_response = model.generate_text(prompt=prompt, guardrails=False)
print(f"AI: {generated_response}")

AI:  جميع الكلمات المذكورة لا تحتوي على حرف الألف بشكل مباشر. ومع ذلك، إذا كنت تقصد وجود الحرف "أ" كجزء من بعض الكلمات المركبة أو المشتقة منها (مثل استخدام كلمة "بيت" كصفة)، فإن هذا ليس قاعدة ثابتة ويمكن أن تختلف حسب السياق والاستخدام اللغوي.

إذا كانت لديك كلمات أخرى ترغب في تحليلها ومعرفة ما إذا كانت تحتوي على حرف الألف أم لا، يرجى طرحها وسأكون سعيداً لمساعدتك. 


In [ ]:
demo_template2 = PromptTemplate (
    template = " ما هي الكلمة التي تحتوي على حرف ال{letteer2}" | 
)

In [296]:
examples = [
    {
        "السؤال": "ما هي الكلمة التي تحتوي على حرف الألف بغض النظر عن موقعه: سيارة، رسم، عسل، نوم؟",
        " الإجابة الصحيحة": "سيارة"
    },
     {
        "السؤال": "ما هي الكلمة التي تحتوي على حرف الألف بغض النظر عن موقعه: أرنب ، ثعلب ، سفر ، نوم؟",
        "الإجابة الصحيحة ": "أرنب "

    },
    {
        "السؤال": "ما هي الكلمة التي تحتوي على حرف الباء  بغض النظر عن موقعه: بالون ، حديقة ، ذرة ، سمكة ؟",
        "الإجابة الصحيحة ": "بالون  "

    },
    {
        "السؤال": "ما هي الكلمة التي تحتوي على حرف الباء  بغض النظر عن موقعه: طائرة ،  فلفل، قبعة ، فيل ؟",
        "الإجابة الصحيحة ": "قبعة  "

    },
   {
        "السؤال": "ما هي الكلمة التي تحتوي على حرف الباء  بغض النظر عن موقعه: سنارة ،  عشب ،  خفاش ، وردة ؟",
        "الإجابة الصحيحة ": "عشب "

    },
      {
        "السؤال": "ما هي الكلمة التي تحتوي على حرف التاء  بغض النظر عن موقعه: ثعبان ،  فلفل،  ضفدع ، تاج ؟",
        "الإجابة الصحيحة ": "تاج  "

    },
    {
        "السؤال": "ما هي الكلمة التي تحتوي على حرف التاء  بغض النظر عن موقعه: ظرف ، كتاب ،  قلب ، مسجد ؟",
        "الإجابة الصحيحة ": "كتاب  "

    },
    {
        "السؤال": "ما هي الكلمة التي تحتوي على حرف التاء  بغض النظر عن موقعه:  كلب ، شمس ،  بنت ، جمل  ؟",
        "الإجابة الصحيحة ": "بنت  "

    },
    {
        "السؤال": "ما هي الكلمة التي تحتوي على حرف التاء  بغض النظر عن موقعه: سلحفاة ، ثوم ،  صقر ، قرد  ؟",
        "الإجابة الصحيحة ": "سلحفاة  "

    },
    {
        "السؤال": "ما هي الكلمة التي تحتوي على حرف التاء  بغض النظر عن موقعه: خاروف ، تمر ،  زهور ، نافذة  ؟",
        "الإجابة الصحيحة ": "نافذة  "

    }
]




In [367]:
from langchain.prompts import PromptTemplate

# تعريف البرومبت مع أمثلة
template = PromptTemplate(
    template="""
        
    """,
    input_variables=["الحرف"]
)

# إدخال جديد لتوليد السؤال
input_data = {
    "الحرف": "الطاء"  # هنا يمكنك تغيير الحرف حسب الحاجة
}
generated_prompt = template.format(**input_data)
generated_prompt
generated_prompt = f"""<s> [INST] {generated_prompt} [/INST]"""
prompt = f"""{generated_prompt}"""
generated_response = model.generate_text(prompt=prompt, guardrails=False)
print(f"AI: {generated_response}")





AI:  أنا مساعد ذكاء اصطناعي، كيف يمكنني مساعدتك اليوم؟ إذا كان لديك سؤال أو استفسار، لا تتردد في طرحه وسأكون سعيداً لمساعدتك. 


In [ ]:
الكود الي تحت مشكلته اولا الاجابه قبل الاسئله . تانيا بس موقف على حرف الجيم  بلش يهنق 

In [224]:
from langchain.prompts import FewShotPromptTemplate
examples = [
    {
        "letter": "أ",
        "question": "اختر الكلمة التي تحتوي على الحرف 'أ':",
        "correct_word": "سيارة",
        "options": "سيارة، رسم، عسل، نوم"
    },
    {
        "letter": "أ",
        "question": "اختر الكلمة التي تحتوي على الحرف 'أ':",
        "correct_word": "أرنب",
        "options": "أرنب، ثعلب، سفر، نوم"
    },
    {
        "letter": "ب",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ب':",
        "correct_word": "بالون",
        "options": "بالون، حديقة، ذرة، سمكة"
    },
    {
        "letter": "ب",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ب':",
        "correct_word": "قبعة",
        "options": "طائرة، فلفل، قبعة، فيل"
    },
    {
        "letter": "ب",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ب':",
        "correct_word": "عشب",
        "options": "سنارة، عشب، خفاش، وردة"
    },
    {
        "letter": "ت",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ت':",
        "correct_word": "تاج",
        "options": "ثعبان، فلفل، ضفدع، تاج"
    },
    {
        "letter": "ت",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ت':",
        "correct_word": "كتاب",
        "options": "ظرف، كتاب، قلب، مسجد"
    },
    {
        "letter": "ت",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ت':",
        "correct_word": "بنت",
        "options": "كلب، شمس، بنت، جمل"
    },
    {
        "letter": "ت",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ت':",
        "correct_word": "سلحفاة",
        "options": "سلحفاة، ثوم، صقر، قرد"
    },
    {
        "letter": "ت",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ت':",
        "correct_word": "نافذة",
        "options": "خاروف، تمر، زهور، نافذة"
    }
]
example_prompt = PromptTemplate(
    input_variables=["letter", "question", "correct_word", "options"],
    template="الحرف: {letter}\nالسؤال: {question}\nالكلمة الصحيحة: {correct_word}\nالخيارات: {options}\n")

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="استخدم الأمثلة التالية لتوليد سؤال جديد وخيارات تتضمن الكلمة  الصحيحة والكلمات الخاطئة.",
    suffix="الحرف: {letter}\nالسؤال: {question}\nالخيارات: {options}",
    input_variables=["letter", "question", "options"]
)

letter_to_test = "س"
question_template = "اختر الكلمة التي تحتوي على الحرف '{letter}':"
formatted_prompt = prompt.format(
    letter=letter_to_test,
    question=question_template,
    options="حدد النموذج تلقائيًا الخيارات التي تتضمن الكلمة الصحيحة وأخطاء مماثلة"
)
formatted_prompt = f"""<s> [INST] {formatted_prompt} [/INST]"""
prompt = f"""{formatted_prompt}"""
generated_response = model.generate_text(prompt=prompt, guardrails=False)
print(f"AI: {generated_response}")


AI:  الحرف: س
السؤال: اختر الكلمة التي تحتوي على الحرف 'س':
الكلمة الصحيحة: سرير
الخيارات: سرير، كرسي، باب، شباك

أو يمكنك تغيير السؤال والخيارات حسب رغبتك لتتناسب مع متطلباتك التعليمية أو التدريبية. 


In [337]:
letter = "خ"

In [15]:

from langchain.prompts import FewShotPromptTemplate, PromptTemplate


examples = [
    {
        "letter": "أ",
        "question": "اختر الكلمة التي تحتوي على الحرف 'أ':",
        "correct_word": "سيارة",
        "options": "سيارة، رسم، عسل، نوم"
    },
    {
        "letter": "أ",
        "question": "اختر الكلمة التي تحتوي على الحرف 'أ':",
        "correct_word": "أرنب",
        "options": "أرنب، ثعلب، سفر، نوم"
    },
    {
        "letter": "ب",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ب':",
        "correct_word": "بالون",
        "options": "بالون، حديقة، ذرة، سمكة"
    },
    {
        "letter": "ب",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ب':",
        "correct_word": "قبعة",
        "options": "طائرة، فلفل، قبعة، فيل"
    },
    {
        "letter": "ب",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ب':",
        "correct_word": "عشب",
        "options": "سنارة، عشب، خفاش، وردة"
    },
    {
        "letter": "ت",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ت':",
        "correct_word": "تاج",
        "options": "ثعبان، فلفل، ضفدع، تاج"
    },
    {
        "letter": "ت",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ت':",
        "correct_word": "كتاب",
        "options": "ظرف، كتاب، قلب، مسجد"
    },
    {
        "letter": "ت",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ت':",
        "correct_word": "بنت",
        "options": "كلب، شمس، بنت، جمل"
    },
    {
        "letter": "ت",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ت':",
        "correct_word": "سلحفاة",
        "options": "سلحفاة، ثوم، صقر، قرد"
    },
    {
        "letter": "ت",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ت':",
        "correct_word": "نافذة",
        "options": "خاروف، تمر، زهور، نافذة"
    }
]

# إعداد القالب لتضمين الأمثلة أولا، ثم إجابة السؤال بشكل منفصل في الأسفل
example_prompt = PromptTemplate(
    input_variables=["letter", "question", "correct_word", "options"],
    template="الحرف: {letter}\nالسؤال: {question}\nالكلمة الصحيحة: {correct_word}\nالخيارات: {options}\n"
)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="استخدم الأمثلة التالية لتوليد سؤال جديد وخيارات تتضمن كلمة واحدةصحيحة والكلمات الخاطئة.\n",
    suffix="\nالآن، بناءً على الأمثلة، اجعل الحرف '{letter}' واختر السؤال المطلوب:\nالسؤال: {question}\nالخيارات: {options}",
    input_variables=["letter", "question", "options"]
)

# الحرف الذي تريد استخدامه والسؤال مع ترك الخيارات لتوليدها
letter_to_test = "ق"
question_template = f"اختر الكلمة التي تحتوي على الحرف '{letter_to_test}':"

formatted_prompt = prompt.format(
    letter=letter_to_test,
    question=question_template,
    options="حدد النموذج تلقائيًا الخيارات التي تتضمن الكلمة الصحيحة وأخطاء مماثلة"
)

# تنسيق النموذج في رسالة إلى النموذج
formatted_prompt = f"""<s> [INST] {formatted_prompt} [/INST]"""
prompt_text = f"""{formatted_prompt}"""

# توليد النص من النموذج
generated_response = model.generate_text(prompt=prompt_text, guardrails=False)

# عرض النتيجة
print(f"AI: {generated_response}")

ConnectTimeout: _ssl.c:989: The handshake operation timed out

In [ ]:
letter = " س"
examples = [
    {
        "letter": "أ",
        "question": "اختر الكلمة التي تحتوي على الحرف 'أ':",
        "correct_word": "سيارة",
        "options": "سيارة، رسم، عسل، نوم"
    },
    {
        "letter": "أ",
        "question": "اختر الكلمة التي تحتوي على الحرف 'أ':",
        "correct_word": "أرنب",
        "options": "أرنب، ثعلب، سفر، نوم"
    },
    {
        "letter": "ب",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ب':",
        "correct_word": "بالون",
        "options": "بالون، حديقة، ذرة، سمكة"
    },
    {
        "letter": "ب",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ب':",
        "correct_word": "قبعة",
        "options": "طائرة، فلفل، قبعة، فيل"
    },
    {
        "letter": "ب",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ب':",
        "correct_word": "عشب",
        "options": "سنارة، عشب، خفاش، وردة"
    },
    {
        "letter": "ت",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ت':",
        "correct_word": "تاج",
        "options": "ثعبان، فلفل، ضفدع، تاج"
    },
    {
        "letter": "ت",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ت':",
        "correct_word": "كتاب",
        "options": "ظرف، كتاب، قلب، مسجد"
    },
    {
        "letter": "ت",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ت':",
        "correct_word": "بنت",
        "options": "كلب، شمس، بنت، جمل"
    },
    {
        "letter": "ت",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ت':",
        "correct_word": "سلحفاة",
        "options": "سلحفاة، ثوم، صقر، قرد"
    },
    {
        "letter": "ت",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ت':",
        "correct_word": "نافذة",
        "options": "خاروف، تمر، زهور، نافذة"
    }
]

TEMPLATE_ARABIC = """
بصفتي معلمًا في المرحلة الابتدائية، أقوم بتصميم نشاط تعليمي ممتع لمساعدة طفل صغير على التعرف على الحروف.
لكل حرف، سأقوم بإنشاء خمسة أسئلة اختيار من متعدد لتعزيز التعرف على الحروف.
سأختار حرفًا معينًا وهو "{letter}" وأعد كل سؤال مع أربعة خيارات للكلمات،\
    حيث تبدأ  كلمة واحدة فقط بالحرف "{letter}".

    هذه بعض الامثلة على كيفية صياغة الأسئلة لاحد الحروف وليس المختار: {examples}.
"""
generation_prompt_arabic = PromptTemplate(
    input_variables=["letter", "question", "correct_word", "options"],
    template=TEMPLATE_ARABIC,
)
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=generation_prompt_arabic,
    prefix="استخدم الأمثلة التالية لتوليد سؤال جديد وخيارات تتضمن كلمة واحدصحيحة والكلمات الخاطئة.\n",
    suffix="\nالآن، بناءً على الأمثلة، اجعل الحرف '{letter}' واختر السؤال المطلوب:\nالسؤال: {question}\nالخيارات: {options}",
    input_variables=["letter", "question", "options"]
)
letter = "م"

formatted_prompt = f"""<s> [INST] {formatted_prompt} [/INST]"""
prompt_text = f"""{formatted_prompt}"""


generated_response = model.generate_text(prompt=prompt_text, guardrails=False)


print(f"AI: {generated_response}")

AI:  [/

في هذا المثال، سأقوم بإنشاء سؤال يتضمن حرف "ق" وسأقدم خيارات متعددة للاختيار من بينها. يرجى ملاحظة أن الكلمات قد تكون صحيحة أو خاطئة حسب ما تختاره أنت كمستخدم. إليك مثال للسؤال والخيارات المتعلقة بحرف "ق":

السؤال: اختر الكلمة التي تحتوي على الحرف "ق":
الكلمة الصحيحة: قمر
الخيارات: قمر, قرش, قلم, قطار

يمكنك تعديل هذه الخيارات وإضافة كلمات أخرى إذا كنت ترغب في ذلك. بعد اختيار المستخدم للكلمة الصحيحة (مثل "قمر")، يمكن للنظام تقديم ردود فعل تؤكد صحة الاختيار أو تشير إلى أنه يجب المحاولة مرة أخرى مع خيار آخر. يمكنك استخدام نفس الأسلوب لإنشاء أسئلة جديدة وحروف مختلفة لتوسيع قاعدة المعرفة وتحسين مهارات اللغة للمستخدمين. 


In [43]:
letter3="ل"

the question


In [44]:
TEMPLATE_story =f"""<<SYS>>

أنت شخصية تدعى "حلم" اللطيفة، معلمة تساعد الأطفال على تعلم اللغة العربية. تقوم  بتصميم نشاط تعليمي ممتع لمساعدة طفل صغير على التعرف على الحرف الذي اختاره الطفل {letter}
<</SYS>>
<<SYS>>
قم بإنشاء ثلاثة أسئلة اختيار من متعدد لتعزيز التعرف على الحروف 
سيختار الطفل  حرفًا معينًا وهو "{letter3}" وأعد كل سؤال مع أربع خيارات للكلمات،
حيث تحتوي كلمة واحدة فقط  بالحرف "{letter3}".<</SYS>> 
<<SYS>> [INST] ضع في الخيارات كلمه واحده تحتوي على الحرف الذي اختاره الطفل  {letter3}[/INST]  <</SYS>> 

"<<SYS>>" [INST] {letter3}"   هذه  بعض الامثلة و اعمل على توليد  ثلاثة اسئله مشابهه بالحرف الذي اختاره الطفل مع وضع الاجابه الصحيحة بعد طرح السؤال 
الحرف: "أ"
السؤال: "اختر الكلمة التي تحتوي على الحرف 'أ':"
الخيارات: "سيارة، رسم، عسل، نوم"
الكلمة الصحيحة: "سيارة".لانها الكلمة الوحيده من الاربع  كلمات التي تحتوي على حرف الالف و موقعه  في وسط الكلمة 
كلمة رسم هي الكلمة الخطا لانها لا تحتوي على حرف الالف 

الحرف: "أ"
السؤال: "اختر الكلمة التي تحتوي على الحرف 'أ':"
الخيارات: "أرنب، ثعلب، سفر، نوم"
الكلمة الصحيحة: "أرنب" . لانها الكلمة الوحيدة  من الاربع كلمات التي تحتوي على حرف الالف  و موقعه في بدايه الكلمة 

الحرف: "ب"
السؤال: "اختر الكلمة التي تحتوي على الحرف 'ب':"
الخيارات: "بالون، حديقة، ذرة، سمكة"
الكلمة الصحيحة: "بالون". لانها الكلمة الوحيدة من الاربع  كلمات التي  تحتوي على حرف  الباء و موقعه  في بدايه الكلمة 

الحرف: "ب"
السؤال: "اختر الكلمة التي تحتوي على الحرف 'ب':"
الخيارات: "طائرة، فلفل، قبعة، فيل"
الكلمة الصحيحة: "قبعة". لانها الكلمة الوحيدة من الاربع كلمات التي تحتوي حرف الباء و موقعه  في  وسط الكلمة 


الحرف: "ب"
السؤال: "اختر الكلمة التي تحتوي على الحرف 'ب':"
الخيارات: "سنارة، عشب، خفاش، وردة"
الكلمة الصحيحة: "عشب". لانها الكلمه الوحيده من الاربع كلمات التي تحتوي على حرف الباء و موقعه في اخر الكلمة 
[/INST]<</SYS>>
""" 
letter3 = "ل"
TEMPLATE_story = TEMPLATE_story.format(letter = letter3)

generated_response = model.generate_text(prompt=TEMPLATE_story, guardrails=False)
print(f"AI: {generated_response}")


c:\Users\u1046\AppData\Local\anaconda3\Lib\site-packages\ibm_watsonx_ai\foundation_models\inference\base_model_inference.py:723: UserWarning: Parameters [random_state ] is/are not recognized and will be ignored.
  warnings.warn(


AI: هذه الأسئلة مصممة خصيصاً للحرف "ل":

1. اختر الكلمة التي تحتوي على الحرف "ل":
    - سيارة
    - رسم
    - عسل
    - نوم
الإجابة الصحيحة: سيارة

2. اختر الكلمة التي تحتوي على الحرف "ل":
    - أرنب
    - ثعلب
    - سفر
    - نوم
الإجابة الصحيحة: أرنب

3. اختر الكلمة التي تحتوي على الحرف "ل":
    - بالون
    - حديقة
    - ذرة
    - سمكة
الإجابة الصحيحة: بالون

4. اختر الكلمة التي تحتوي على الحرف "ل":
    - طائرة
    - فلفل
    - قبعة
    - فيل
الإجابة الصحيحة: قبعة

5. اختر الكلمة التي تحتوي على الحرف "ل":
    - سنارة
    - عشب
    - خفاش
    - وردة
الإجابة الصحيحة: عشب

ت


تجربة الفيو شوت تيمليت 

In [45]:
examples = [
    {
        "letter": "أ",
        "question": "اختر الكلمة التي تحتوي على الحرف 'أ':",
        "options": "سيارة، رسم، عسل، نوم",
        "correct_word": "سيارة"
    },
    {
        "letter": "أ",
        "question": "اختر الكلمة التي تحتوي على الحرف 'أ':",
        "options": "أرنب، ثعلب، سفر، نوم",
        "correct_word": "أرنب"
    },
    {
        "letter": "ب",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ب':",
        "options": "بالون، حديقة، ذرة، سمكة",
         "correct_word": "بالون"
    },
    {
        "letter": "ب",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ب':",
        "options": "طائرة، فلفل، قبعة، فيل",
        "correct_word": "قبعة"
    },
    {
        "letter": "ب",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ب':",
        "options": "سنارة، عشب، خفاش، وردة",
        "correct_word": "عشب"
    },
    {
        "letter": "ت",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ت':",
        "options": "ثعبان، فلفل، ضفدع، تاج",
        "correct_word": "تاج"
    },
    {
        "letter": "ت",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ت':",
        "options": "ظرف، كتاب، قلب، مسجد",
        "correct_word": "كتاب"
    },
    {
        "letter": "ت",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ت':",
        "options": "كلب، شمس، بنت، جمل",
        "correct_word": "بنت"
        
    },
    {
        "letter": "ت",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ت':",
        "options": "سلحفاة، ثوم، صقر، قرد",
        "correct_word": "سلحفاة"
        
    },
    {
        "letter": "ت",
        "question": "اختر الكلمة التي تحتوي على الحرف 'ت':",
        "options": "خاروف، سماء، زهور، نافذة",
        "correct_word": "نافذة"
        
    }
]


In [46]:
example_prompt = PromptTemplate(
    input_variables=["letter", "question", "options", "correct_word"],
    template=""" 
    - الحرف: "{letter}"
      - السؤال: "{question}"
      - الخيارات: "{options}"
      - الكلمة الصحيحة: "{correct_word}"[/INST] <</SYS>>
    """
)

In [209]:
letter3 = "ل"

In [47]:
few_shot_prompt = FewShotPromptTemplate(
    examples=examples, 
    example_prompt=example_prompt, 
    prefix="""<<SYS>>
أنت شخصية تدعى "حلم" اللطيفة، معلمة تساعد الأطفال على تعلم اللغة العربية. تقوم  بتصميم نشاط تعليمي ممتع لمساعدة طفل صغير على التعرف على الحرف الذي اختاره الطفل {letter}
<</SYS>>
<<SYS>>[INST]
قم بإنشاء ثلاثة أسئلة اختيار من متعدد لتعزيز التعرف على الحرف {letter}
 وأعد كل سؤال مع أربع خيارات للكلمات،
حيث تحتوي كلمة واحدة فقط  بالحرف "{letter}".<</SYS>> 
<<SYS>> [INST] ضع في الخيارات كلمه واحده تحتوي على الحرف الذي اختاره الطفل  {letter}[/INST]  <</SYS>>""",  
    suffix="""
الحرف: "{letter}"
السؤال: "اختر الكلمة التي تحتوي على الحرف '{letter}':"
الخيارات: """,  
    input_variables=["letter"]
)


In [48]:
letter = "ف"
prompt = few_shot_prompt.format(letter=letter)
print(prompt)

generated_response = model.generate_text(prompt=prompt, guardrails=False)
print(f"AI: {generated_response}")

<<SYS>>
أنت شخصية تدعى "حلم" اللطيفة، معلمة تساعد الأطفال على تعلم اللغة العربية. تقوم  بتصميم نشاط تعليمي ممتع لمساعدة طفل صغير على التعرف على الحرف الذي اختاره الطفل ف
<</SYS>>
<<SYS>>[INST]
قم بإنشاء ثلاثة أسئلة اختيار من متعدد لتعزيز التعرف على الحرف ف
 وأعد كل سؤال مع أربع خيارات للكلمات،
حيث تحتوي كلمة واحدة فقط  بالحرف "ف".<</SYS>> 
<<SYS>> [INST] ضع في الخيارات كلمه واحده تحتوي على الحرف الذي اختاره الطفل  ف[/INST]  <</SYS>>

 
    - الحرف: "أ"
      - السؤال: "اختر الكلمة التي تحتوي على الحرف 'أ':"
      - الخيارات: "سيارة، رسم، عسل، نوم"
      - الكلمة الصحيحة: "سيارة"[/INST] <</SYS>>
    

 
    - الحرف: "أ"
      - السؤال: "اختر الكلمة التي تحتوي على الحرف 'أ':"
      - الخيارات: "أرنب، ثعلب، سفر، نوم"
      - الكلمة الصحيحة: "أرنب"[/INST] <</SYS>>
    

 
    - الحرف: "ب"
      - السؤال: "اختر الكلمة التي تحتوي على الحرف 'ب':"
      - الخيارات: "بالون، حديقة، ذرة، سمكة"
      - الكلمة الصحيحة: "بالون"[/INST] <</SYS>>
    

 
    - الحرف: "ب"
      - السؤال: "اختر الكلمة ال

In [178]:

generated_response = model.generate_text(prompt=promt, guardrails=False)
print(f"AI: {generated_response}")

AI: هذه الأسئلة تعزز فهم الطفل للحروف المختارة وتساعده على تحديد الكلمات المناسبة بشكل صحيح. يمكنك استخدام هذا النموذج لإنشاء المزيد من الأسئلة لحروف أخرى أيضًا! تذكر أن تختار كلمات بسيطة ومناسبة لعمر الطفل ومستوى معرفته باللغة. 


In [169]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate


In [ ]:
tryit = """<<SYS>> [INST]
أنت شخصية تدعى "حلم" اللطيفة، معلمة تساعد الأطفال على تعلم اللغة العربية. تقوم بتصميم نشاط تعليمي ممتع لمساعدة طفل صغير على التعرف على الحرف الذي اختاره الطفل {letter}
[/INST]<</SYS>>
<<SYS>>[INST]سيختار الطفل  حرفًا معينًا وهو "{letter}"
بعد اختيار الطفل الحرف {letter},
قم بإنشاء ثلاثة أسئلة اختيار من متعدد من  ثلاثة خيارات للكلمات  بحيث كلمة واحد فقط تحتوي على الحرف {letter}
.[/INST]<</SYS>> 
 """